<a href="https://colab.research.google.com/github/AtufaShireen/Hackathons/blob/main/hypd/content%2Bcollaborative_HYPD_CHALLENGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Reading Data

In [47]:
import json
f = open('/content/drive/MyDrive/hypd/content_masked.json')
 
# returns JSON object as
# a dictionary
content = json.load(f)



In [3]:
len(content)

7581

In [4]:
content[7]

{'_id': {'$oid': '*****************bbdc'},
 'brand_ids': [{'$oid': '*****************1058'}],
 'caption': 'Keeps you active and energetic throughout the day\n#fast&up #workout #supplement #energy',
 'catalog_ids': [{'$oid': '5f92b20f40006497ff42106b'},
  {'$oid': '5f9d63bb3ac6177434f48029'},
  {'$oid': '5f93e4a040006497ff4210dd'}],
 'catalog_info': None,
 'category_path': ['/615d825d528b706ef3882b2b/615ea667528b706ef3883322'],
 'comment_count': 0,
 'created_at': {'$date': '2021-04-21T21:35:05.719Z'},
 'hashtags': ['#fast', '#workout', '#supplement', '#energy'],
 'influencer_ids': [{'$oid': '*****************033e'}],
 'is_active': True,
 'is_processed': True,
 'label': {'age_groups': ['26-30', '30-35'],
  'genders': ['M', 'F'],
  'interests': ['workout',
   'gym',
   'vitamin',
   'energy',
   'protien',
   'hydration']},
 'last_sync': {'$date': '2022-02-12T09:34:43.513Z'},
 'like_count': 4,
 'like_ids': [{'$oid': '*****************bee1'},
  {'$oid': '*****************0f4b'},
  {'$oid':

In [42]:
import json
f = open('/content/drive/MyDrive/hypd/catalog_masked.json')
 
# returns JSON object as
# a dictionary
catalog = json.load(f)



In [6]:
catalog[6]

{'_id': {'$oid': '*****************de5d'},
 'base_price': 1949,
 'brand_id': {'$oid': '*****************ddc1'},
 'cat_one': '*****************7324',
 'cat_three': '*****************f077',
 'cat_two': '*****************7cf9',
 'keywords': ['dress',
  'dresses',
  'maxi dress',
  'maxi',
  'black dress',
  'Embroidered dress',
  'summer dress',
  'black',
  'women',
  'girls',
  'girl',
  'western',
  'western wear',
  'casual',
  'Casuals ',
  'clothing',
  'apparel',
  'apparels',
  'trendy',
  'stylish',
  'brand',
  'vritta'],
 'name': 'Floral Embroidered Maxi Dress - Black',
 'retail_price': 1750,
 'status': 'Publish'}

# Product Recommendation To Influencers post

## Data Collection

* Extracting these values from content.json and creating a dataframe.

    * catalog_ids
    * hashtags
    * interests
    * genders
    * caption

* Extracting these values from catalog.json and creating a dataframe.

    * catalog_id
    * name
    * keywords

In [7]:
no_vals = 0
content_dict = {
    'catalog_id':[],
    'hashtags':[],
    'interests':[],
    'genders':[],
    'caption':[]
}
for i in range(1,len(content)-1):
    # print(i)
    # print("i-",i)
    if content[i]['catalog_ids']!=None:
        content_dict['catalog_id'].append([j['$oid'] for j in content[i]['catalog_ids']])
        if content[i]['hashtags']!=None:
            content_dict['hashtags'].append([k.replace('#','') for k in content[i]['hashtags']])
        else:
            content_dict['hashtags'].append([None])
   
        if content[i]['label']!=None:
            if content[i]['label']['interests']!=None:
                content_dict['interests'].append(content[i]['label']['interests'])
            else:
                content_dict['interests'].append(None)
            genders = content[i]['label']['genders']
            if genders!=None:
                for j in range(len(genders)):
                    if genders[j].lower()=='f':
                        genders[j]='girl'
                    elif genders[j].lower()=='m':
                        genders[j]='boy'
                content_dict['genders'].append(genders)
            else:
                content_dict['genders'].append([None])
        else:
            no_vals+=1
            content_dict['genders'].append([None])
            content_dict['interests'].append([None])
        content_dict['caption'].append( content[i]['caption'] )

In [8]:
print(no_vals)

420


In [9]:
print(len(content_dict['hashtags']))
print(len(content_dict['catalog_id']))
print(len(content_dict['interests']))
print(len(content_dict['genders']))
print(len(content_dict['caption']))
print(len(content_dict))

5843
5843
5843
5843
5843
5


* arrays lens should be equal to create df of dict of arrays.

In [10]:
content_df = pd.DataFrame(content_dict)
content_df.head(10)

,catalog_id,hashtags,interests,genders,caption
0,[6035106e58b8f4136b6e4505],"[beardoil, package, gift, sample, pebbles]","[Sample, testers, free products]","[boy, girl]",A curated box of surprise samples from select ...
1,"[60094d9352e78c76b023fc9b, 6008556952e78c76b02...","[quaclothing, workwear, corporatefashion, offi...","[work wear, work, women, office look, professi...",[girl],Ways to Wear Black Cotton Trousers\n\nThe tape...
2,"[5fbbbb65c94eac4d3ca460cd, 5fbcc096c94eac4d3ca...","[kicaactive, workout, gymwear, activewear]","[work out, activewear, gym, workout, body toni...",[girl],She's anything but ordinary. Wearing Kica Acti...
3,[5fda294b0a30d919d2439684],"[destello, fashion, trending, ootd, shirt]","[shirt, graphic shirt, quirky, orinted shirt, ...",[boy],Midweek feels. There's so much more to a week ...
4,"[602a1a177f064048a3e7457d, 602a0df67f064048a3e...","[mensclothing, floralprint, printedshirt, beac...","[tailored , custom, printed shirts, casual, ev...",[boy],"Vibrant Tropical Shirt from 15 buttons, styled..."
5,"[5f92e8fc40006497ff4210a3, 5f93cebf40006497ff4...","[fast, supplements, energy, gym]","[work out, gym, supplements, protien, energy, ...",[boy],Experience absolute beast mode during workouts...
6,"[5f92b20f40006497ff42106b, 5f9d63bb3ac6177434f...","[fast, workout, supplement, energy]","[workout, gym, vitamin, energy, protien, hydra...","[boy, girl]",Keeps you active and energetic throughout the ...
7,[5fbb77d8c94eac4d3ca46091],"[prolixrbeauty, prolixr, skincare, hydration]","[skincare, beauty, makeup, looking good, routi...",[girl],Keep your skin hydrated with @prolixr All-New ...
8,[5ff585d21beb3405bd1a3ec1],"[kicaactive, leggings, fitness, workoutwear]","[kica active, kica, fitness, gym wear, active ...",[girl],"A hypnotising flow, and nope we aren’t going t..."
9,[5ffebac16a21ca0e6cdd316d],"[halffullhalfempty, spaghettidress, ootd, fash...","[striped dress, casual dress, dress, western w...",[girl],Dresses with pockets >> ! 😍\n#halffullhalfempt...


In [11]:

content_df.isnull().sum().sum()

0

In [43]:
catalog_dict = {
    'catalog_id':[],
    'name':[],
    'keywords':[]
}
for i in range(len(catalog)):
    catalog_dict['catalog_id'].append(catalog[i]['_id']['$oid'][-4:])
    catalog_dict['name'].append(catalog[i]['name'])
    catalog_dict['keywords'].append(catalog[i]['keywords'])


In [44]:
catalog_df = pd.DataFrame(catalog_dict)
catalog_df.head(10)

,catalog_id,name,keywords
0,93d0,Moisturising Cream | Shea Butter & Vitamin E,"[cream, moisturising, shea, butter, vitamin e,..."
1,94cb,Bold Swimsuit,"[swimsuit, gymwear, active wear, swimming cost..."
2,93ff,Onion Seed Hair Oil,"[oil, hair fall, growth oil, hair, onion, blac..."
3,9548,Maroon Embroidered Shift Dress,"[one piece, casual, party, long dress, dress]"
4,9548,Maroon Embroidered Shift Dress,"[one piece, casual, party, long dress, dress]"
5,dcfd,Grey Crop Top With Embroidered Sleeves,"[top, casual wear, trendy, women wear]"
6,de5d,Floral Embroidered Maxi Dress - Black,"[dress, dresses, maxi dress, maxi, black dress..."
7,df1a,Engy Blue Stripe Set,"[coat, coat pant set, office wear, formal wear..."
8,f00d,Taupe Printed Dress With Tie-Up Belt,"[one peice, casual wear, fit and flare dress,..."
9,dd35,Winona Stripe Jumpsuit,"[women apparel , ladies jumpsuit, strip jumpsu..."


In [14]:
catalog_df.isnull().sum().sum()

0

## Preprocessing

* Lower texts
* preprocessing text (only for captions)

    * tokenizing
    * removing special characters
    * removing stop words (added few extra words to nltk stop words like today,tommorow etc.,)

* combining these text from content and creating a set (for input)
    * hashtags
    * interests
    * genders
    * caption
    * catalog_tags

In [15]:
content_df['hashtags'] = content_df['hashtags'].apply(lambda x: [i.lower() for i in x if i!=None])
content_df['interests'] = content_df['interests'].apply(lambda x: [i.lower() for i in x if i!=None])
content_df['caption'] = content_df['caption'].apply(lambda x: x.lower() if (x!=None) else '')


In [16]:
content_df.iloc[1]['caption']

'ways to wear black cotton trousers\n\nthe tapered pair is cut to a clean, streamlined silhouette with slanted pockets and natural wood buttons. they’re cut from durable cotton woven with stretch so the slim-leg silhouette is flexible. we show you how to style these black pants in 4 different ways!\n#quaclothing #workwear #corporatefashion #officelook'

In [17]:

stop_words = stopwords.words('english')
stop_words.extend(['get','today','tommorow','yesterday'])

def preprocess_sent(x):
    x = x.lower()
    words = word_tokenize(x)
    # words = [char for char in words if char not in string.punctuation]
    words = [char for char in words if char.isalpha()]

    words = [word for word in words if word not in stop_words]
    return words


In [18]:
content_df['caption'] = content_df['caption'].apply(lambda x: preprocess_sent(x))

In [19]:
def unite(x):
    tags = x['hashtags']
    # print(tags)
    inter = x['interests']
    # print(inter)
    gen = x['genders']
    cap = x['caption']

    h = []
    h.extend(tags)
    h.extend(inter)
    h.extend(gen)
    h.extend(cap)
    return list(set(h))


In [20]:
content_df['new_tags'] = content_df.apply(unite,axis=1)

In [21]:
train_df = content_df.drop(['hashtags','interests','genders','caption'],axis=1)
train_df.head()

,catalog_id,new_tags
0,[6035106e58b8f4136b6e4505],"[beardoil, hypd, dash, getting, girl, happy, p..."
1,"[60094d9352e78c76b023fc9b, 6008556952e78c76b02...","[different, women, buttons, stretch, durable, ..."
2,"[5fbbbb65c94eac4d3ca460cd, 5fbcc096c94eac4d3ca...","[gymwear, gym wear, kicaactive, work out, girl..."
3,[5fda294b0a30d919d2439684],"[casual, much, shirt, graphic shirt, orinted s..."
4,"[602a1a177f064048a3e7457d, 602a0df67f064048a3e...","[floralprint, buttons, custom, carefully, plan..."


* adding the catalog name to the contents (using catalog dataset, and last 4 unmasked characters of catalog_ids).

In [22]:
train_df = train_df.explode('catalog_id')
train_df.head()

,catalog_id,new_tags
0,6035106e58b8f4136b6e4505,"[beardoil, hypd, dash, getting, girl, happy, p..."
1,60094d9352e78c76b023fc9b,"[different, women, buttons, stretch, durable, ..."
1,6008556952e78c76b023fbed,"[different, women, buttons, stretch, durable, ..."
1,6009486752e78c76b023fc8b,"[different, women, buttons, stretch, durable, ..."
2,5fbbbb65c94eac4d3ca460cd,"[gymwear, gym wear, kicaactive, work out, girl..."


In [23]:
train_df.isnull().sum()
train_df.dropna(axis=0,inplace=True)

In [24]:
def cat_name(cat_ids):
    bad_ids = []
    cat_names = []
    for i in range(len(catalog)):
        names = []
        cat = catalog[i]['_id']
        # print('cat',cat)
        if cat!=None:
            if cat['$oid'][-4:]==cat_ids[-4:]:
                try:
                    cat_names.append(catalog[i]['name'])
                except Exception as e:
                    print(e)
                    cat_names.append(None)
        else:
            print('cat is none')
    return list(set(cat_names))

def flatten_list(lists):
    flat_list = []
    # Iterate through the outer list
    for element in lists:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return list(set(flat_list))

def cat_tags(cat_ids):
    bad_ids = []
    cat_names = []
    for i in range(len(catalog)):
        names = []
        cat = catalog[i]['_id']
        # print('cat',cat)
        if cat!=None:
            if cat['$oid'][-4:]==cat_ids[-4:]:
                cat_names.append(catalog[i]['keywords'])
        else:
            print('cat is none')
    return flatten_list(cat_names)


In [25]:

train_df['catalog_name'] = train_df['catalog_id'].apply(lambda x: cat_name(x))
train_df['catalog_tags'] = train_df['catalog_id'].apply(lambda x: cat_tags(x))

In [26]:
train_df.head()

,catalog_id,new_tags,catalog_name,catalog_tags
0,6035106e58b8f4136b6e4505,"[beardoil, hypd, dash, getting, girl, happy, p...",[Look Under The Pebbles 🥰],"[pebble, pebbles, easter, sample, surprise, sa..."
1,60094d9352e78c76b023fc9b,"[different, women, buttons, stretch, durable, ...",[Smart Cotton Crop Trousers - Black],"[women, trendy, fashionista , designer , brand..."
1,6008556952e78c76b023fbed,"[different, women, buttons, stretch, durable, ...",[Ruched Poplin Blouson - Black],"[women, trendy, fashionista , fashionable , br..."
1,6009486752e78c76b023fc8b,"[different, women, buttons, stretch, durable, ...",[Stretch Wool Tailored Suit - Grey ],"[women, trendy, fashionable , brand, style, of..."
2,5fbbbb65c94eac4d3ca460cd,"[gymwear, gym wear, kicaactive, work out, girl...",[Cheer Leggings],"[activewear, gym, fitness, gymwear, leggings]"


In [27]:
train_df['catalog_name'].iloc[4]

['Cheer Leggings']

In [28]:
train_df.isnull().sum()

catalog_id      0
new_tags        0
catalog_name    0
catalog_tags    0
dtype: int64

* Got the names for every catalog_id, great.

In [29]:
def unite_tags(x):
    new_tags = x['new_tags']
    # print(tags)
    catalog_name = x['catalog_name']
    # print(inter)
    catalog_tags = x['catalog_tags']

    h = []
    h.extend(new_tags)
    h.extend(catalog_name)
    h.extend(catalog_tags)
    return list(set(h))

In [30]:
train_df['catalog_new_tags'] = train_df.apply(unite_tags,axis=1)
# train_df.drop(['new_tags','catalog_tags'],axis=1,inplace=True)
# train_df.head()

## Modelling

* creating corpus

In [31]:
train_df['catalog_name'] = train_df['catalog_name'].apply(lambda x: ' '.join(i for i in x))
train_df.head()

,catalog_id,new_tags,catalog_name,catalog_tags,catalog_new_tags
0,6035106e58b8f4136b6e4505,"[beardoil, hypd, dash, getting, girl, happy, p...",Look Under The Pebbles 🥰,"[pebble, pebbles, easter, sample, surprise, sa...","[Look Under The Pebbles 🥰, beardoil, hypd, das..."
1,60094d9352e78c76b023fc9b,"[different, women, buttons, stretch, durable, ...",Smart Cotton Crop Trousers - Black,"[women, trendy, fashionista , designer , brand...","[women, buttons, trendy, fashionista , stretch..."
1,6008556952e78c76b023fbed,"[different, women, buttons, stretch, durable, ...",Ruched Poplin Blouson - Black,"[women, trendy, fashionista , fashionable , br...","[women, buttons, trendy, fashionista , stretch..."
1,6009486752e78c76b023fc8b,"[different, women, buttons, stretch, durable, ...",Stretch Wool Tailored Suit - Grey,"[women, trendy, fashionable , brand, style, of...","[women, buttons, trendy, stretch, durable, fas..."
2,5fbbbb65c94eac4d3ca460cd,"[gymwear, gym wear, kicaactive, work out, girl...",Cheer Leggings,"[activewear, gym, fitness, gymwear, leggings]","[gymwear, gym wear, kicaactive, work out, girl..."


In [32]:
print('calalog name \n',train_df.iloc[4]['catalog_name'])
print('calalog tags \n',train_df.iloc[4]['catalog_new_tags'])

calalog name 
 Cheer Leggings
calalog tags 
 ['gymwear', 'gym wear', 'kicaactive', 'work out', 'girl', 'kica', 'body toning', 'earthy', ' gymwear', 'looking', 'pure', 'activewear', 'anything', 'wearing', 'leggings', 'perfection', 'Cheer Leggings', 'gym', 'fitness', 'like', 'ordinary', 'active', 'rusty', 'workout', 'colours']


In [50]:
train_df.shape

(13132, 5)

In [51]:
new_train = train_df.copy()
new_train.drop(['catalog_tags','new_tags'],axis=1,inplace=True)
new_train['catalog_new_tags_'] = new_train['catalog_new_tags'].apply(lambda x: ' '.join([str(i) for i in x]))
new_train = new_train[~new_train.duplicated(subset=['catalog_id','catalog_new_tags_'],keep='first')]

In [52]:
print(new_train.shape)
print(train_df.shape)

(13108, 4)
(13132, 5)


## TFIDF

### Training

In [56]:
new_train['catalog_new_tags']

0       [Look Under The Pebbles 🥰, beardoil, hypd, das...
1       [women, buttons, trendy, fashionista , stretch...
1       [women, buttons, trendy, fashionista , stretch...
1       [women, buttons, trendy, stretch, durable, fas...
2       [gymwear, gym wear, kicaactive, work out, girl...
                              ...                        
5841    [facemist, lasting, earth, beauty , function, ...
5841    [facemist, lasting, Shikakai, moisture, earth,...
5842    [lipbalm , lasting, earth, function, kajal , f...
5842    [lipbalm , glossy, women, lasting, trendy, tri...
5842    [lipbalm , tint, family, glossy, women, bright...
Name: catalog_new_tags, Length: 13108, dtype: object

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TF-IDF feature extractor
tfidf = TfidfVectorizer()
tfidf.fit(new_train['catalog_new_tags_'])
tfidf_recipe = tfidf.transform(new_train['catalog_new_tags_'])

In [68]:
tfidf_recipe

<13108x11859 sparse matrix of type '<class 'numpy.float64'>'
	with 643714 stored elements in Compressed Sparse Row format>

In [72]:
def get_recommendations(n_recs,rec):
    ingredients_tfidf = tfidf.transform([rec])
    cos_sim = map(lambda x: cosine_similarity(ingredients_tfidf, x), tfidf_recipe)
    scores = list(cos_sim)
    new_df = new_train[['catalog_id','catalog_name']].copy()
    new_df['scores'] = scores
    new_df['scores'] = new_df['scores'].apply(lambda x: x[0][0])
    new_df = new_df.drop_duplicates(subset=['catalog_name'])
    new_df = new_df.sort_values(by='scores',ascending=False)[:n_recs]
    return new_df

# a = '''
# qua stylish pockets workwear suit cut boss lady trendy slanted silhouette cool suits different streamlined girl clean
#  corporatefashion stretch grey suit designer work wear pair tapered fashionable  blazer office wear work girls flexible
#   woven trousers formals Stretch Wool Tailored Suit - Grey  women ways brand
#  cotton durable black buttons formal show pants wear quaclothing office look western professional
#   boss lady look fashion  wood style trouser fashionista natural officelook western wear
# '''
# get_recommendations(5,a)

### Prediction

In [ ]:
content[8]

In [74]:
caption = 'Keep your skin hydrated with @prolixr All-New Vitamin-C Serum💥 #prolixrbeauty #prolixr #skincare #hydration'
interests =['skincare',
   'beauty',
   'makeup',
   'looking good',
   'routine',
   'vitamin c',
   'prolixr']
genders = ['girl']
tags = ['prolixrbeauty', 'prolixr', 'skincare', 'hydration']
caption = preprocess_sent(caption)

h = []
h.extend(interests)
h.extend(caption)
h.extend(genders)
h.extend(tags)
h = list(set(h))


In [125]:
input_ = ' '.join(i for i in h)
get_recommendations(5,input_)

,catalog_id,catalog_name,scores
7,5fbb77d8c94eac4d3ca46091,Beauty Shield Vitamin C Serum,0.632338
433,5f5def66b35d64d69f1b93e7,Vitamin C Face Serum,0.163210
1705,60117f8a52e78c76b023fec5,Perfect Skin Bundle Kit,0.148477
572,6107a33f8a16dc68df17e427,Vitamin C Professional Face Serum,0.142155
572,6105426e8a16dc68df17e3a3,Vitamin C Brightening Face Wash,0.138250


## KNN

### Training

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [ ]:
knn1 = KNeighborsClassifier()
knn1.fit(tfidf_recipe,new_train['catalog_name'])

KNeighborsClassifier()

### prediction

In [ ]:
caption = 'Keep your skin hydrated'
interests =['skincare',
   'beauty',
   'makeup',
   'looking good',
   'routine',
   'vitamin c',
   'prolixr']
genders = ['girl']
tags = ['prolixrbeauty', 'prolixr', 'skincare', 'hydration']
caption = preprocess_sent(caption)

h = []
h.extend(interests)
h.extend(caption)
h.extend(genders)
h.extend(tags)
h = list(set(h))


In [ ]:
new = tfidf.transform(h)

In [ ]:
knn1.predict(new)

array(['Pout Potion Liquid Matte Lipstick - Girl Boss',
       'Good On You Hydra Matte Lipstick - Wyld Rose', '',
       ' Coffee & Love Scrub', 'Arz Kiya Hai Unisex Hoodie',
       'Arz Kiya Hai Unisex Hoodie', 'Very Berry Moisturiser',
       'Beauty Shield Vitamin C Serum', '',
       'Aqua Marine Foaming Cleanser', '', '',
       'Beauty Shield Vitamin C Serum'], dtype=object)

## Content Recommendation for users

In [173]:
content[0]

{'_id': {'$oid': '*****************b634'},
 'brand_ids': [{'$oid': '*****************be8b'}],
 'caption': 'Test pebble',
 'catalog_ids': [{'$oid': '5f6362aa6e9f136645ab2d34'}],
 'catalog_info': None,
 'category_path': None,
 'comment_count': 0,
 'created_at': {'$date': '2021-04-19T09:47:58.735Z'},
 'hashtags': None,
 'influencer_ids': [{'$oid': '*****************4055'}],
 'is_active': False,
 'is_processed': True,
 'label': {'age_groups': ['26-30', '30-35'],
  'genders': ['M', 'F'],
  'interests': ['interest']},
 'last_sync': None,
 'like_count': 0,
 'like_ids': None,
 'liked_by': None,
 'media_type': 'video',
 'processed_at': {'$date': '2021-04-19T09:48:42.575Z'},
 'series_ids': None,
 'sync': None,
 'sync_type': None,
 'type': 'pebble',
 'view_count': 0}

In [180]:
infs =[]
for i in range(len(content)):
    if content[i]['influencer_ids']!=None: 
        user_id = [k['$oid'][-4:] for k in content[i]['influencer_ids']]
        infs.append(user_id)
g = []
for i in infs:
    for j in i:
        g.append(j)
print(len(g))
print(len(set(g)))
unqinfs = list(set(g))

7687
1786


In [262]:
lis = []
for i in infs_matrix['interests']:
    for j in i:
        lis.append(j)

In [193]:
infs_matrix = {
    'influencer_id':[],
    'like_count':[],
    'comment_count':[],
    'view_count':[],
    'interests':[],
    'gender':[],
    'catalog_ids':[]
}

for i in range(len(unqinfs)):
    
    for j in range(len(content)):
            if unqinfs[i] in [k['$oid'][-4:] for k in content[j]['influencer_ids']]:
                infs_matrix['influencer_id'].append(unqinfs[i])
                infs_matrix['like_count'].append(content[i]['like_count'])
                infs_matrix['comment_count'].append(content[i]['comment_count'])
                infs_matrix['view_count'].append(content[i]['view_count'])
                infs_matrix['interests'].append(content[i]['label']['interests'])
                infs_matrix['gender'].append(content[i]['label']['genders'])
                if content[j]['catalog_ids']!=None:
                    infs_matrix['catalog_ids'].append([g['$oid'][-4:] for g in content[j]['catalog_ids']])
                else:
                    infs_matrix['catalog_ids'].append('None')
with open("infs_matrix.json", "w") as outfile:
    json.dump(infs_matrix, outfile)

In [190]:
users =[]
for i in range(len(content)):
    if content[i]['like_count']!=0: 
        user_id = [k['$oid'][-4:] for k in content[i]['liked_by']]
        users.append(user_id)
g = []
for i in users:
    for j in i:
        g.append(j[-4:])
print(len(g))
print(len(set(g)))
unqusers = list(set(g))

73358
16723


In [191]:
users_matrix = {
    'user_id':[],
    'content_id':[],
    'influencer_ids':[],
    'catalog_ids':[]

}
for i in range(len(unqusers)):
    
    for j in range(len(content)):
        if content[j]['like_count']!=0:
            
            if unqusers[i] in [k['$oid'][-4:] for k in content[j]['liked_by']]:
                users_matrix['user_id'].append(unqusers[i])
                users_matrix['content_id'].append(content[j]['_id']['$oid'][-4:])
                users_matrix['influencer_ids'].append([g['$oid'][-4:] for g in content[j]['influencer_ids']])
                if content[j]['catalog_ids']!=None:
                    users_matrix['catalog_ids'].append([g['$oid'][-4:] for g in content[j]['catalog_ids']])
                else:
                    users_matrix['catalog_ids'].append('None')
with open("user_matrix.json", "w") as outfile:
    json.dump(users_matrix, outfile)

In [192]:
for k in users_matrix.keys():
    print(len(users_matrix[k]))

print('---')
for k in inf.keys():
    print(len(inf[k]))

73176
73176
73176
73176


In [289]:
users_df = pd.DataFrame(users_matrix)
users_df.head()

,user_id,content_id,influencer_ids,catalog_ids
0,fdd9,91cc,"[faed, 17c8]","[998f, 997d, e0df, 9983, e167]"
1,fdd9,249a,[8642],"[c220, c112]"
2,fdd9,2589,[8642],"[1123, 111c, 117f]"
3,fdd9,3082,[02bd],[1f49]
4,7e2d,26c2,[23bf],None


In [290]:
user_df = users_df.explode('content_id')
# user_df = users_df.explode('influencer_ids')
# user_df = users_df.explode('catalog_ids')
user_df.head()

,user_id,content_id,influencer_ids,catalog_ids
0,fdd9,91cc,"[faed, 17c8]","[998f, 997d, e0df, 9983, e167]"
1,fdd9,249a,[8642],"[c220, c112]"
2,fdd9,2589,[8642],"[1123, 111c, 117f]"
3,fdd9,3082,[02bd],[1f49]
4,7e2d,26c2,[23bf],None


In [301]:

def user_factor1(content_ids):
    # user factor 1
    # interests : male female
    intersts = {'boy':0,'girl':0}
    for j in range(len(content_ids)):
        for i in range(len(content)):
            if content[i]['_id']['$oid'][-4:] == content_ids[j]:
                if content[i]['label']:
                    print(content[i]['label']['genders'])
                    for k in content[i]['label']['genders']:
                        if k.lower() == 'm' or k.lower()=='boy' or k.lower()=='male' or k.lower()=='men' or k.lower()=='man':
                            intersts['boy']+=1
                        else:
                            intersts['girl']+=1
    return intersts


In [291]:
user_df[user_df['user_id'] == '7e2d']

,user_id,content_id,influencer_ids,catalog_ids
4,7e2d,26c2,[23bf],None
5,7e2d,21cc,[033e],None


In [ ]:
users_ = {
    'id':[],
    'score_f':[],
    'score_m':[],
    'content':[]
}
for i_ in list(set(user_df['user_id'])):
    # print(i_)
    df = user_df[user_df['user_id'] == i_]
    users_['id'].append(i_)
    cont = list(set(df['content_id']))
    # print(cont)
    users_['content'].append(cont)
    score = user_factor1(cont)
    users_['score_f'].append(score['girl'])
    users_['score_m'].append(score['boy'])

In [337]:
user_df = pd.DataFrame({'user_id':users_['id'],
                        'user_f':users_['score_f'],
                        'user_m':users_['score_m'],
                        'content_id':users_['content']})
user_df = user_df.explode('content_id')

In [338]:
user_df.head()

,user_id,user_f,user_m,content_id
0,7e2d,3,0,26c2
0,7e2d,3,0,21cc
1,fdd9,3,1,3082
1,fdd9,3,1,249a
1,fdd9,3,1,2589


##### deriving factors for user to like the post.

In [339]:
import re
def content_factor_f(content_id):
    # how much the content is for specfic gender
    k = catalog_df[catalog_df['catalog_id']==content_id]['keywords'].values
    flat_list = [x for xs in k for x in xs]
    f = list(filter(lambda v: re.match('women|womens|girl|girls|female|females', v), flat_list))
    if len(f)!=0:
        return 1
    else:
        return 0

import re
def content_factor_m(content_id):
    # how much the content is for specfic gender
    k = catalog_df[catalog_df['catalog_id']==content_id]['keywords'].values
    flat_list = [x for xs in k for x in xs]
    m = list(filter(lambda v: re.match('men|man|boy|boys|male|males', v), flat_list))
    if len(m)!=0:
        return 1
    else:
        return 0

In [341]:
user_df['content_f'] = user_df['content_id'].apply(lambda x: content_factor_f(x))
user_df['content_m'] = user_df['content_id'].apply(lambda x: content_factor_m(x))

In [344]:
user_df.head()

,user_id,user_f,user_m,content_id,content_f,content_m
0,7e2d,3,0,26c2,1,0
0,7e2d,3,0,21cc,0,0
1,fdd9,3,1,3082,0,0
1,fdd9,3,1,249a,0,0
1,fdd9,3,1,2589,0,0


In [376]:
user_df.shape

(73170, 6)

In [23]:
# user_df = pd.read_csv("user_content_df.csv")

In [24]:
usr_df = user_df.copy()

In [25]:
def change_fig(df):
    if df['user_f'] ==0 and df['user_m'] == 0:
        df['user_f'] = 1
        df['user_m'] = 1
    if df['content_f'] ==0 and df['content_m'] == 0:
        df['content_f'] = 1
        df['content_m'] = 1
    return df

In [26]:
usr_df = usr_df.apply(lambda x: change_fig(x),axis=1)

In [34]:
usr_df.iloc[50:60]

,user_id,content_id,score_
50,8cc6,b6fa,0.017331
51,df2b,62f9,0.069324
52,df2b,c08c,0.051993
53,df2b,bfd7,0.051993
54,11d5,6a7f,0.398614
55,11d5,1c7f,0.398614
56,11d5,591d,0.398614
57,11d5,47c8,0.398614
58,11d5,ca36,0.398614
59,11d5,9b47,0.398614


In [29]:
def score(df):
    score = 0
    if df['user_f'] >0 and df['content_f'] >0:
        score += df['user_f']
    if df['user_m'] >0 and df['content_m'] >0:
        score += df['user_m']
    return score
usr_df['score'] = usr_df.apply(lambda x: score(x),axis=1)

In [30]:
usr_df['score_'] = ((usr_df['score'] - min(usr_df['score'])) / max(usr_df['score']) - min(usr_df['score'])) *10

In [32]:
usr_df.drop(['user_f','user_m','content_f','content_m','score'],axis=1,inplace=True)

In [33]:
usrs_df = usr_df.copy()

# SVD
R: user matrix

U: user feature matrix (how much user likes the feature)

Sigma: diagonal matrix

V: item feature matrix (content of item)


In [36]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
usrs_df.head(
)

,user_id,content_id,score_
0,7e2d,26c2,0.051993
1,7e2d,21cc,0.051993
2,fdd9,3082,0.069324
3,fdd9,249a,0.069324
4,fdd9,2589,0.069324


In [38]:
from surprise import Reader,Dataset, SVD,NormalPredictor,Prediction
from surprise.model_selection import cross_validate
from collections import defaultdict

reader = Reader(rating_scale=(0.5,5))
data = Dataset.load_from_df(usrs_df[['user_id','content_id','score_']],reader)
algo=SVD()
cross_validate(algo,data,measures=['RMSE'],cv=5,verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6391  0.6296  0.6433  0.6326  0.6342  0.6358  0.0049  
Fit time          3.48    6.20    8.91    7.95    3.53    6.01    2.23    
Test time         0.23    0.34    0.34    0.25    0.10    0.25    0.09    


{'fit_time': (3.4786906242370605,
  6.199813604354858,
  8.905590772628784,
  7.948318004608154,
  3.5254905223846436),
 'test_rmse': array([0.6391443 , 0.62962856, 0.64331769, 0.63261647, 0.63418783]),
 'test_time': (0.23139286041259766,
  0.3419368267059326,
  0.3436920642852783,
  0.2549149990081787,
  0.09651637077331543)}

In [39]:
from surprise.model_selection import GridSearchCV
param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

0.6650719651489758
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [40]:
algo = gs.best_estimator['rmse']
trainset = data.build_full_trainset()
algo.fit(trainset)

In [45]:
catalog_df.drop_duplicates(subset = ['catalog_id'],inplace=True,keep='first')
catalog_df['content_id']=catalog_df['catalog_id']
catalog_df.head()

,catalog_id,name,keywords,content_id
0,93d0,Moisturising Cream | Shea Butter & Vitamin E,"[cream, moisturising, shea, butter, vitamin e,...",93d0
1,94cb,Bold Swimsuit,"[swimsuit, gymwear, active wear, swimming cost...",94cb
2,93ff,Onion Seed Hair Oil,"[oil, hair fall, growth oil, hair, onion, blac...",93ff
3,9548,Maroon Embroidered Shift Dress,"[one piece, casual, party, long dress, dress]",9548
5,dcfd,Grey Crop Top With Embroidered Sleeves,"[top, casual wear, trendy, women wear]",dcfd


In [48]:
def find_cat(x):
    s = []
    names = []
    for i in range(len(content)):
        if content[i]['_id']['$oid'][-4:] == x:
            if content[i]['catalog_ids']:
                s.append([g['$oid'] for g in content[i]['catalog_ids']])
    
    s = list(set([x for xs in s for x in xs]))
    for t in s:
        f = catalog_df[catalog_df['catalog_id'] == t[-4:]]['name']
        names.append(f)
    return names
find_cat('3082')

[1965    Defender For Her Complete Grooming Kit
 Name: name, dtype: object]

In [49]:
# df_data  = pd.merge(usrs_df,catalog_df,how='left', on='content_id')
# df_data[df_data['user_id']=='fdd9'].sort_values('score_')[:10]
df_data = usrs_df.copy()
df_data['catalog_name']= df_data['content_id'].apply(lambda x: find_cat(x))

In [50]:
df_data

,user_id,content_id,score_,catalog_name
0,7e2d,26c2,0.051993,[[Peach Nude Pink Lip & Cheek Tint]]
1,7e2d,21cc,0.051993,[]
2,fdd9,3082,0.069324,[[Defender For Her Complete Grooming Kit]]
3,fdd9,249a,0.069324,"[[Black Maxi Dress ], [Solid Khaki Jumpsuit]]"
4,fdd9,2589,0.069324,"[[Yellow Halter Shirt], [Black Drama Co-Ord Se..."
...,...,...,...,...
73165,f089,7ea7,0.086655,"[[Good Morning- Nail Paint Remover], [Ibiza Su..."
73166,f089,ccc1,0.086655,[[Pout Potion Liquid Matte Lipstick - Drama Qu...
73167,f089,59de,0.086655,[[Plant-based Chicken Nuggets]]
73168,f089,7e41,0.069324,[]


In [51]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)


In [ ]:
top_n